In [1]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle

In [2]:
!head train.csv

0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,240,636,-1,1,694,-3,7,665,-1,2,650,-3,2,671,0,3,696,-1,4,692,-1,5,647,2,7,4
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,240,636,-1,1,694,-3,7,665,-1,2,650,-3,2,671,0,3,696,-1,4,692,-1,5,647,2,7,4,0,240,635,-1,1,688,-3,7,663,-1,2,648,-3,2,668,0,3,691,-1,4,687,-1,5,640,2,7,4
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,240,636,-1,1,694,-3,7,665,-1,2,650,-3,2,671,0,3,696,-1,4,692,-1,5,647,2,7,4,0,240,635,-1,1,688,-3,7,663,-1,2,648,-3,2,668,0,3,691,-1,4,687,-1,5,640,2,7,4,8,248,634,-1,1,685,-3,7,663,-1,2,648,-3,2,666,0,3,685,-1,4,680,-1,5,633,2,7,1
0,240,636,-1,1,694,-3,7,665,-1,2,650,-3,2,671,0,3,696,-1,4,692,-1,5,647,2,7,4,0,240,635,-1,1,688,-3,7,663,-1,2,648,-3,2,668,0,3,691,-1,4,687,-1,5,640,2,7,4,8,248,634,-1,1,685,-3,7,663,-1,2,648,-3,2,666,0,3,68

In [3]:
features = ["f" + str(i) for i in range(0,107)]
label = ["action"]

headers = features + label
print(len(headers))

df = pd.read_csv('train.csv', 
                 sep = ',', 
                 header = None,
                 names = headers)
len(df)

108


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f98,f99,f100,f101,f102,f103,f104,f105,f106,action
0,0,0,0,0,0,0,0,0,0,0,...,696,-1,4,692,-1,5,647,2,7,4
1,0,0,0,0,0,0,0,0,0,0,...,691,-1,4,687,-1,5,640,2,7,4
2,0,0,0,0,0,0,0,0,0,0,...,685,-1,4,680,-1,5,633,2,7,1
3,0,240,636,-1,1,694,-3,7,665,-1,...,679,-1,4,674,-1,5,627,2,7,1
4,0,240,635,-1,1,688,-3,7,663,-1,...,674,-1,4,667,-1,5,621,2,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,8,89,456,2,2,96,-1,2,574,0,...,638,0,1,649,1,1,605,-1,1,2
836,8,89,455,2,2,94,-1,2,573,0,...,637,0,1,648,1,1,604,-1,1,5
837,8,89,454,2,2,91,-1,2,572,0,...,637,0,1,650,1,1,606,-1,1,0
838,8,89,453,2,2,89,-1,2,571,0,...,636,0,1,650,1,1,605,-1,1,5


In [4]:
len(df)

840

In [ ]:
# import pandas_profiling
# pandas_profiling.ProfileReport(df)

Basic logisting regresion

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25)

classifier = LogisticRegression().fit(X_train, y_train)
y_pred_test = classifier.predict(X_test)
y_pred_train = classifier.predict(X_train)

test_accuracy = accuracy_score(y_test, y_pred_test)
train_accuracy = accuracy_score(y_train, y_pred_train)
print('Training accuracy: {0:.2f}, evaluation accuracy: {1:.2f}'.format(train_accuracy, test_accuracy))

/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Training accuracy: 0.75, evaluation accuracy: 0.55


/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


xgboost

In [6]:
y_train = df["action"]
X_train = df.drop(columns="action")

model = XGBClassifier().fit(X_train, y_train)

model.evals_result

<bound method XGBClassifier.evals_result of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)>

random forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(random_state=1, n_estimators=1000)
forest_model.fit(X_train, y_train)

In [ ]:
pickle.dump(forest_model, open("ai_model_forest.pkl","wb"))

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                    hidden_layer_sizes=(128, 128, 128), random_state=1)

mlp_model.fit(X_train, y_train)   

In [ ]:
pickle.dump(mlp_model, open("ai_model_mlp.pkl","wb"))

In [ ]:
# y_train = df[0]
# X_train = df.drop(columns=[0])
# y_train_matrix = y_train.values
# X_train_matrix = X_train.values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns="action"), df["action"], 
                                                    test_size=0.2, random_state=7)

xgb_model = XGBClassifier(n_estimators=1000).fit(X_train, y_train,eval_set = [[X_test, y_test]])

In [ ]:
pickle.dump(xgb_model, open("ai_model_xgb.pkl","wb"))

In [ ]:
!ls

In [7]:
# model_p = pickle.load(open("ai_model.pkl","rb"))

model.predict(np.array([8,280,630,-1,1,674,-3,7,667,-1,2,649,-3,2,662,0,3,662,-1,4,655,-1,5,608,2,7,1,8,288,630,-1,1,672,-3,7,668,-1,2,650,-3,2,661,0,3,657,-1,4,649,-1,5,602,2,7,1,8,296,629,-1,1,670,-3,7,669,-1,2,651,-3,2,660,0,3,651,-1,4,643,-1,5,596,2,7,1,8,296,628,-1,1,665,-3,7,667,-1,2,650,-3,2,658,0,3,647,-1,4,638,-1,5,589,2,7]).reshape(1, -1))

array([4])